In [3]:
import numpy as np 
import matplotlib.pyplot as plt 
import gymnasium as gym      
from gymnasium import * 
import time
import seaborn as sns
import random

In [31]:
class FootballEnv(gym.Env):
    
    def __init__(self, grid_rows=51,grid_cols=121):
        super(FootballEnv, self).__init__()
        self.grid_rows = grid_rows
        self.grid_cols = grid_cols

        self.action_space = spaces.Discrete(11) #0-left 1-down 2-right 3-up 4-pass 5-shoot 6-pick_ball

        self.observation_space = spaces.Discrete(grid_rows*grid_cols)

        self.layout = np.zeros((grid_rows,grid_cols),dtype=str)
        self.layout[:, :] = "."
        self.layout[self.grid_rows//2, self.grid_cols//2] = "C"
        self.layout[self.grid_rows//2-8 : self.grid_rows//2+9, -6:-1] = "D"
        self.layout[:, -1] = "O"
        self.layout[:, 0] = "O"
        self.layout[0, :] = "O"
        self.layout[-1, :] = "O"
        self.layout[self.grid_rows//2-4 : self.grid_rows//2+5, -1] = "G"
        self.layout[:, self.grid_cols//2] = "M"

        self.state = (self.grid_rows-1,0)

        self.rewards = np.zeros((grid_rows,grid_cols))
        self.rewards[self.layout == "G"] = 100
        self.rewards[self.layout == "."] = -1
        self.rewards[self.layout == "*"] = 1
        
        self.ball_position = (self.grid_rows//2,self.grid_cols//2)
        self.player_position = (np.random.randint(0,self.grid_rows-1),np.random.randint(0,self.grid_cols-1))
        self.goal_post1 = "G"
        self.layout[self.ball_position[0], self.ball_position[1]] = "B"
        self.layout[self.player_position[0], self.player_position[1]] = "P"
        
        
    def _get_state_index(self):
        return self.state[0] * self.grid_cols + self.state[1]
    
    def step(self,action):
        x,y = self.state

        if action == 0: #left
            y = max(0, y-1) 
        elif action == 1: #down
            x = min(self.grid_rows-1, x+1)
        elif action == 2: #Right 
            y = min(self.grid_cols-1, y+1)
        elif action == 3:
            x = max(0, x-1)
        elif action == 4:
            x = max(0, x-1)
            y = max(0, y-1)
            
        elif action == 5:
            x = max(0, x-1)
            y = min(self.grid_cols-1, y+1)
        elif action == 6:
            x = min(self.grid_rows-1, x+1)
            y = max(0, y-1)
        
        elif action == 7:
            x = min(self.grid_rows-1, x+1)
            y = min(self.grid_cols-1, y+1)

        self.state = (x,y)  #State Updation

        reward = self.rewards[x,y]

        done =  self.layout[x,y] == 1 or self.layout[x,y] == 2

        return self._get_state_index(),reward,done,{}
    
    def reset(self):
        self.state = (self.grid_rows-1,0)
        return self._get_state_index()
    
    def render(self):
        grid = np.full(self.layout.shape, '.', dtype=str)
        grid[self.layout == "C"] = 'C'
        grid[self.layout == "G"] = 'G'
        grid[self.layout == "B"] = 'B'
        grid[self.layout == "P"] = 'P'
        grid[self.layout == 'M'] = "|"
        grid[self.layout == 'D'] = "*"
        grid[self.layout == "O"] = "o"
        row, col = self.state
        
        for row in grid:
            print(end=' ')
            for cell in row:
                print(cell, end=' ')
            print()


In [32]:
env = FootballEnv()
env.reset()
env.render()

 o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o | o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o o 
 o . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . | . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . o 
 o . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . | . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . o 
 o . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . | . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . o 
 o . . . . . . . . . . .